51CTO课程频道：http://edu.51cto.com/lecturer/index/user_id-12330098.html<br>
优酷频道：http://i.youku.com/sdxxqbf<br>
微信公众号：深度学习与神经网络<br>
Github：https://github.com/Qinbf<br>

In [1]:
import pandas as pd
from tqdm import tqdm
import re
import numpy as np
from six.moves import xrange

In [2]:
topic_info = pd.read_table("./ieee_zhihu_cup/topic_info.txt",sep='\t',header=None)
print(topic_info.iloc[0:5])

                     0                                                  1  \
0   738845194850773558                               -5833678375673307423   
1  3738968195649774859                                2027693463582123305   
2  4738849194894773882                                1127459907694805235   
3  7739004195693774975  2904932941037075699,1160326435131345730,725917...   
4 -7261194805221226386                               -5833678375673307423   

                  2     3                                                  4  \
0             c0,c1    w0  c0,c1,c2,c3,c4,c5,c6,c7,c0,c1,c8,c9,c10,c11,c1...   
1           c39,c40   w24  c41,c42,c43,c39,c40,c4,c44,c45,c46,c47,c48,c49...   
2    c172,c31,c0,c1  w102                                                NaN   
3   c39,c40,c5,c173  w103  c39,c40,c23,c21,c174,c74,c5,c173,c17,c35,c39,c...   
4  c36,c31,c45,c237  w148                                          c238,c239   

                                                   5  
0

In [3]:
# 话题字典
topic_dict = {}
for i in xrange(topic_info.shape[0]):
    topic_dict[i] = topic_info.iloc[i][0]

In [4]:
predict = open('predict.txt', "r")
examples = predict.readlines()
text = np.array([line.split(" ") for line in examples])

In [5]:
label = []
for line in tqdm(text):
    num2label = []
    for i in xrange(5):
        num2label.append(topic_dict[int(line[i])]) # 把0-1999编号转成原来的id
    label.append(num2label)
label = np.array(label)

100%|██████████| 217360/217360 [00:01<00:00, 160389.86it/s]


In [6]:
np.savetxt("temp.txt",label,fmt='%d')

In [7]:
def clean_str(string):
    string = re.sub(r" ", ",", string)
    return string

file1 = open('temp.txt', "r")
examples = file1.readlines()
examples = [clean_str(line) for line in examples]
file1.close()

file1 = open('temp.txt', "w")
file1.writelines(examples)
file1.close()

In [8]:
# predict文件导入
predict_file = 'temp.txt'
predict_reader = pd.read_table(predict_file,sep=' ',header=None)
print(predict_reader.iloc[0:5])

                                                   0
0  -3517637179126242000,-4653836020042332281,4715...
1  3418451812342379591,2858911571784840089,238291...
2  -7358589937244777363,-5265476641576484497,7477...
3  -7046289575185911002,-4653836020042332281,-587...
4  4715442001886462944,-8963554618409314978,11274...


In [9]:
# 导入question_train_set
eval_reader = pd.read_table('./ieee_zhihu_cup/question_eval_set.txt',sep='\t',header=None)
print(eval_reader.iloc[0:3])

                     0                                                  1  \
0  6215603645409872328  c924,c531,c102,c284,c188,c104,c98,c107,c11,c11...   
1  6649324930261961840  c346,c1549,c413,c294,c675,c504,c183,c74,c541,c...   
2 -4251899610700378615  c96,c97,c97,c98,c99,c100,c101,c141,c42,c42,c10...   

                                                   2  \
0  w1340,w1341,w55,w1344,w58,w6,w24178,w26959,w47...   
1  w40132,w1357,w1556,w1380,w2464,w33,w16791,w109...   
2  w53,w54,w1779,w54,w1309,w54,w369,w949,w65587,w...   

                                                   3  \
0  c1128,c529,c636,c572,c1321,c139,c540,c223,c510...   
1                                                NaN   
2  c149,c148,c148,c42,c185,c95,c95,c186,c186,c186...   

                                                   4  
0  w4094,w1618,w20104,w19234,w1097,w1005,w4228,w2...  
1                                                NaN  
2                                                NaN  


In [10]:
final_predict = pd.concat([eval_reader.ix[:,0],predict_reader],axis=1)
print(final_predict.iloc[0:5])

                     0                                                  0
0  6215603645409872328  -3517637179126242000,-4653836020042332281,4715...
1  6649324930261961840  3418451812342379591,2858911571784840089,238291...
2 -4251899610700378615  -7358589937244777363,-5265476641576484497,7477...
3  6213817087034420233  -7046289575185911002,-4653836020042332281,-587...
4 -8930652370334418373  4715442001886462944,-8963554618409314978,11274...


In [11]:
final_predict.to_csv('temp.txt', header=None, index=None, sep=',')

final_file = open('temp.txt', "r")
final_examples = final_file.readlines()
final_examples = [re.sub(r'"',"",line) for line in final_examples]
final_file.close()

final_file = open('final_predict.csv', "w")
final_file.writelines(final_examples)
final_file.close()